In [ ]:
# Import necessary libraries
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
import assignment_2_2023.msg
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal, PlanningResult
from std_srvs.srv import SetBool
from actionlib_msgs.msg import GoalStatus
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# Initialize ROS node
rospy.init_node('set_target_client')

# Initialize action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
client.wait_for_server()

# Create widgets
pos_label = widgets.Label(value="Current position: ")
vel_label = widgets.Label(value="Current velocity: ")
goal_label = widgets.Label(value="Current goal: ")
new_goal_button = widgets.Button(description="Set new goal")
cancel_goal_button = widgets.Button(description="Cancel current goal")

# Display widgets
display(pos_label)
display(vel_label)
display(goal_label)
display(new_goal_button)
display(cancel_goal_button)

def update_position_velocity(msg):
    # Update position and velocity labels
    pos_label.value = f"Current position: {msg.pose.pose.position}"
    vel_label.value = f"Current velocity: {msg.twist.twist.linear}, {msg.twist.twist.angular}"

def set_new_goal(button):
    # Get new goal coordinates from user
    input_x = float(input("Enter the x-coordinate for the new goal: "))
    input_y = float(input("Enter the y-coordinate for the new goal: "))

    # Update target position parameters and the goal
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y

    # Send the new goal to the action server
    client.send_goal(goal)
    goal_label.value = f"Current goal: {goal.target_pose.pose.position}"

def cancel_current_goal(button):
    # Cancel the current goal
    client.cancel_goal()
    goal_label.value = "Current goal: None"

# Subscribe to /odom topic
rospy.Subscriber("/odom", Odometry, update_position_velocity)

# Set button click handlers
new_goal_button.on_click(set_new_goal)
cancel_goal_button.on_click(cancel_current_goal)

# Start ROS spin loop
rospy.spin()